<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/algorithms_and_data_structures/sem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3
Сравнительный анализ эффективности методов сортировки данных во внешней памяти.

Требования к функциональным возможностям программы.
Программа должна содержать меню, позволяющее выбирать один из двух режимов работы программы:
1)	сортировка файла данных, сформированных случайным образом;
2)	режим накопления статистических данных
В первом режиме требуется предоставить пользователю следующие возможности:
-	задавать размер числовой последовательности, содержащейся в файле;
-	указывать диапазон значений элементов последовательности;
-	выбирать метод внешней сортировки.
Результаты работы программы в данном режиме:
-	вывести на экран количество сравнений и перестановок элементов массива.
Во втором режиме пользователь должен иметь возможность:
-	выбирать способ формирования элементов последовательности, содержащейся в фай-ле (случайные значения, упорядоченная последовательность значений, значения рас-положены в обратном порядке);
-	задавать диапазон и шаг изменения размера последовательности;
-	выбирать метод внешней сортировки.
Результаты работы программы в данном режиме:
Для каждого значения размера формируется набор данных и сортируется выбранным мето-дом. В файл с указанным именем выводятся значения  времени сортировки для каждого ко-личества элементов в наборе.


Напомним, что
**Для получения оценки «отлично» необходимо программно реализовать следующие методы сортировки данных во внешней памяти: сортировка прямым слиянием; многофазная сортировка. Оценить быстродействие указанных методов и степень естественности их пове-дения.**

In [9]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import csv
import shutil

In [ ]:
def write_csv()


def direct_merge(input_file_name, output_file_name):
    # фаза слияния
    shutil.copy(input_file_name, "A.csv")
    n = 0
    with open(input_file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            if row: # Проверяем, что строка не пустая
                n += 1

    def spliting():
        with open('A.csv', 'r') as a:
            read = csv.reader(a)
            step = 1
            for row in read:
                if step % 2 == 1:
                    with open('B.csv', 'a', newline='') as b:
                        writer = b.writer(b)
                        writer.writeof(row)
                else:
                    with open('C.csv', 'a', newline='') as c:
                        writer = c.writer(c)
                        writer.writeof(row)
                step += 1

    def mergering(window)
            with open('A.csv', 'w', newline='') as a:
                writer = csv.writer(a)

                with open('B.csv', 'w', newline='') as b:
                    reader = csv.reader(b)
                    for row in reader:
                        writer.writerow(row)


    i = 0
    while i <= n:



In [12]:
def create_file(data):
    with open('input.csv', 'w', newline='') as f:
      write = csv.writer(f)
      for element in data:
          write.writerow(element)


def check_file():
    with open('output.csv', 'r') as f:
        read = csv.reader(f)
        l_orig = list(read)
        l_cher = l_orig.copy()
        l_cher.sort()
        if l_cher == l_orig:
            return True
        else:
            return False


In [13]:
d = [[random.randint(1, 1000) for _ in range(11)]]
create_file(d)


In [ ]:
t = []



for i in range(100, 100001, 100):
    data = [random.randint(1, 1000) for _ in range(i)]
    create_file(data)
    time_s = time.time()
    #тут сортировка
    # direct_merge_sort('input.txt', 'output.txt')

    time_f = time.time()
    t.appent(time_f - time_s)




